# Implementation of Basic Homebrew CNN (greyscale)

In [36]:
# General imports 
import sys
import os 
sys.path.insert(1, os.path.join(os.pardir, 'src'))
from itertools import product

# Data imports
import torch
import plotly.express as px
import numpy as np
import mlflow
import matplotlib.pyplot as plt
from torchvision import datasets, transforms

# Homebrew imports 
import model
from utils import one_hot_encode_index
from optimizers import Adam
from activations import Softmax, ReLU
from layers import Dropout, LinearLayer, ConvolutionLayer, PoolingLayer, FlattenLayer
from loss import CategoricalCrossEntropyLoss

## TESTING 
import importlib
importlib.reload(model)
##

<module 'model' from '..\\src\\model.py'>

## Data loaders

In [37]:
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(32),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.Grayscale(num_output_channels=1),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.5],[0.5])
                                      ])

test_transforms = transforms.Compose([transforms.Resize(33),
                                      transforms.CenterCrop(32),
                                      transforms.Grayscale(num_output_channels=1),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.5],[0.5])
                                    ])

# setting up data loaders
data_dir = os.path.join(os.pardir, 'data', 'Plant_leave_diseases_32')

train_data = datasets.ImageFolder(os.path.join(data_dir, 'train'), transform=train_transforms)
test_data = datasets.ImageFolder(os.path.join(data_dir, 'validation'), transform=test_transforms)

### Train config

In [38]:
# Configs 
config = {
    'max_epochs': 10,
    'learning_rate': 0.003,
    'resolution': 32,
    'name': 'Basic_CNN_greyscale_module_homebrew'
}

## Model

In [39]:
mdl = model.Model(Adam(learning_rate=config['learning_rate']),
                      CategoricalCrossEntropyLoss())

# Config early stop 
mdl.add_early_stop(5)

mdl.set_save_config(model_name=config['name'], save_path=os.path.join(os.pardir, 'models'))

# Defining architecture 

mdl.set_sequence([
                    ConvolutionLayer(1, 6, 3),
                    ReLU(),
                    PoolingLayer(6, 2),
                    FlattenLayer(),
                    LinearLayer(1350, 784),
                    ReLU(),
                    LinearLayer(784, 128),
                    ReLU(),
                    LinearLayer(128, 39),
                    Softmax()
                ])
print(mdl)

Model Architecture: 
	 (0): ConvolutionLayer (Trainable: False)
	 (1): ReLU (Trainable: False)
	 (2): PoolingLayer (Trainable: False)
	 (3): FlattenLayer (Trainable: False)
	 (4): LinearLayer (Trainable: True)
	 (5): ReLU (Trainable: False)
	 (6): LinearLayer (Trainable: True)
	 (7): ReLU (Trainable: False)
	 (8): LinearLayer (Trainable: True)
	 (9): Softmax (Trainable: False)



## Training 

In [40]:
mlflow.set_experiment("Plant Leaf Disease")

train_loader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True)
validation_loader = torch.utils.data.DataLoader(test_data, batch_size=128, shuffle=True)


with mlflow.start_run():
    mlflow.log_param('framework', 'homebrew')
    mlflow.log_param('data_split', '90/10')
    mlflow.log_param('type', 'Basic_CNN_greyscale')
    mlflow.log_params(config)
    mdl.train_with_loader(train_loader, epochs=config['max_epochs'], validation_loader=validation_loader, cls_count=39, log_freq=5)

=== Epoch: 1 ===
Step: 0/433, accuracy0.023, loss3.654, learning rate 0.0030000 
Step: 5/433, accuracy0.109, loss3.637, learning rate 0.0030000 
Step: 10/433, accuracy0.070, loss3.630, learning rate 0.0030000 
Step: 15/433, accuracy0.062, loss3.611, learning rate 0.0030000 
Step: 20/433, accuracy0.062, loss3.571, learning rate 0.0030000 
Step: 25/433, accuracy0.070, loss3.594, learning rate 0.0030000 
Step: 30/433, accuracy0.062, loss3.544, learning rate 0.0030000 
Step: 35/433, accuracy0.070, loss3.537, learning rate 0.0030000 
Step: 40/433, accuracy0.125, loss3.393, learning rate 0.0030000 
Step: 45/433, accuracy0.078, loss3.593, learning rate 0.0030000 
Step: 50/433, accuracy0.055, loss3.549, learning rate 0.0030000 
Step: 55/433, accuracy0.078, loss3.540, learning rate 0.0030000 
Step: 60/433, accuracy0.062, loss3.577, learning rate 0.0030000 
Step: 65/433, accuracy0.117, loss3.501, learning rate 0.0030000 
Step: 70/433, accuracy0.094, loss3.600, learning rate 0.0030000 
Step: 75/4

In [ ]:
mdl.layers[3].output.shape

(64, 1350)